<a href="https://colab.research.google.com/github/shuchimishra/Tensorflow_projects/blob/main/Tensorflow_Code/CNN/horses_v_humans_w_Imageaugmentation_w_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation on the Horses or Humans Dataset

In the previous lab, you saw how data augmentation helped improve the model's performance on unseen data. By tweaking the cat and dog training images, the model was able to learn features that are also representative of the validation data. However, applying data augmentation requires good understanding of your dataset. Simply transforming it randomly will not always yield good results.

In the next cells, you will apply the same techniques to the `Horses or Humans` dataset and analyze the results.

In [ ]:
#Install keras-tuner library; uncomment if necessary
!pip install keras-tuner -q

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import RMSprop, SGD, Adam
import keras_tuner
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Download the training set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Download the validation set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

In [ ]:
import os
import zipfile

#Extract the training data
zip_file = 'horse-or-human.zip'
local_handler = zipfile.ZipFile(zip_file,'r')
local_handler.extractall('./horse-or-human')

In [ ]:
#Extract the validation data
zip_file = 'validation-horse-or-human.zip'
local_handler = zipfile.ZipFile(zip_file,'r')
local_handler.extractall('./validation-horse-or-human')

#close the handler
local_handler.close()

In [ ]:
# Directory with training horse pictures
train_horse_dir = os.path.join('horse-or-human','horses')

# Directory with training human pictures
train_human_dir = os.path.join('horse-or-human','humans')

# Directory with validation horse pictures
validation_horse_dir = os.path.join('validation-horse-or-human','horses')

# Directory with validation human pictures
validation_human_dir = os.path.join('validation-horse-or-human','humans')

In [ ]:
print(len(os.listdir(train_horse_dir)))
print(len(os.listdir(train_human_dir)))
print(len(os.listdir(validation_horse_dir)))
print(len(os.listdir(validation_human_dir)))

# **Checking size of image**

In [ ]:
# importing the module
import PIL
from PIL import Image

# loading the image
img = PIL.Image.open(os.path.join(train_horse_dir, os.listdir(train_horse_dir)[9]))

# fetching the dimensions
wid, hgt = img.size
#print the pixels of image
print(wid, hgt)

# **Image Data Augmentation**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#Data Augmentation - Training data
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    rescale=1.0/255.0
)

#Data Augmentation - Validation data
validation_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    './horse-or-human',     # This is the source directory for training images
    target_size=(300, 300), # All images will be resized to 300x300
    class_mode='binary',    # Since we use binary_crossentropy loss, we need binary labels
    batch_size=64
)

# Flow training images in batches of 32 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
    './validation-horse-or-human',  # This is the source directory for training images
    target_size=(300, 300),         # All images will be resized to 300x300
    class_mode='binary',            # Since we use binary_crossentropy loss, we need binary labels
    batch_size=32
)

# **Keras Hyperparameter tuning**

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#Build the model

def build_model(hp):

  inputs = tf.keras.Input(shape=(300, 300, 3))
  x = inputs

  # First Layer
  x = keras.layers.Conv2D(filters=hp.Choice("filters-Conv2d-1", values=[16,32,64,128,256]),kernel_size=(3,3), activation="relu")(x)

  # Tune whether to use MaxPooling2D.
  if hp.Boolean("MaxPooling2D-1"):
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

  # Tune whether to use dropout.
  if hp.Boolean("dropout-1"):
    x = keras.layers.Dropout(rate=hp.Float('dropout-rate-1', min_value=0, max_value=1.0, step=0.1, sampling='linear'))(x)

  #Second Layer
  x = keras.layers.Conv2D(filters = hp.Choice("filters-Conv2d-2", values=[16,32,64,128,256]),kernel_size=(3,3), activation="relu")(x)

  # Tune whether to use MaxPooling2D.
  if hp.Boolean("MaxPooling2D-2"):
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

  # Tune whether to use dropout.
  if hp.Boolean("dropout-2"):
    x = keras.layers.Dropout(rate=hp.Float('dropout-rate-2', min_value=0, max_value=1.0, step=0.1, sampling='linear'))(x)

  #Third Layer
  x = keras.layers.Conv2D(filters = hp.Choice("filters-Conv2d-3", values=[16,32,64,128,256]),kernel_size=(3,3), activation="relu")(x)

  # Tune whether to use MaxPooling2D.
  if hp.Boolean("MaxPooling2D-3"):
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

  # Tune whether to use dropout.
  if hp.Boolean("dropout-3"):
    x = keras.layers.Dropout(rate=hp.Float('dropout-rate-3', min_value=0, max_value=1.0, step=0.1, sampling='linear'))(x)

  #Fourth Layer
  x = keras.layers.Conv2D(filters = hp.Choice("filters-Conv2d-4", values=[16,32,64,128,256]),kernel_size=(3,3), activation="relu")(x)

  # Tune whether to use MaxPooling2D.
  if hp.Boolean("MaxPooling2D-4"):
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

  # Tune whether to use dropout.
  if hp.Boolean("dropout-4"):
    x = keras.layers.Dropout(rate=hp.Float('dropout-rate-4', min_value=0, max_value=1.0, step=0.1, sampling='linear'))(x)

  #Fifth Layer
  x = keras.layers.Conv2D(filters = hp.Choice("filters-Conv2d-5", values=[16,32,64,128,256]),kernel_size=(3,3), activation="relu")(x)

  # Tune whether to use MaxPooling2D.
  if hp.Boolean("MaxPooling2D-5"):
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

  # Tune whether to use dropout.
  if hp.Boolean("dropout-5"):
    x = keras.layers.Dropout(rate=hp.Float('dropout-rate-5', min_value=0, max_value=1.0, step=0.1, sampling='linear'))(x)

  #Mandatory Flatten layer
  x = keras.layers.Flatten()(x)

  # Tune whether to use dropout.
  if hp.Boolean("dropout-6"):
    x = keras.layers.Dropout(rate=hp.Float('dropout-rate-6', min_value=0, max_value=1.0, step=0.1, sampling='linear'))(x)

  # Dense layers
  x = keras.layers.Dense(units=hp.Choice("filters-dense", values=[16,32,64,128,256,512,1024]),activation="relu")(x)
  outputs = keras.layers.Dense(1, activation='sigmoid')(x)

  #Stitch the model
  model_tune = tf.keras.Model(inputs, outputs)

  # Select optimizer
  optimizer=hp.Choice('optimizer', values=['adam', 'RMSprop', 'SGD'])

  # Conditional for each optimizer
  if optimizer == 'adam':
    learning_rate = hp.Float('lrate', min_value=1e-8, max_value=1e-3, sampling='LOG')

  elif optimizer == 'RMSprop':
    learning_rate = hp.Float('lrate', min_value=1e-8, max_value=1e-3, sampling='LOG')

  elif optimizer == 'SGD':
    learning_rate = hp.Float('lrate', min_value=1e-8, max_value=1e-3, sampling='LOG')
    momentum = hp.Float('momentum', min_value=0, max_value=1.0, step=0.1, sampling='linear')

  #compile the model
  model_tune.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy'])

  # model_tune.summary()

  return model_tune

In [ ]:
build_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.BayesianOptimization( #can be Hyperband, RandomSearch, or BayesianOptimization
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    overwrite=True,
    directory="./Model-Tuner",
    project_name="KerasTuning",
)

In [ ]:
#Print summary of search space
tuner.search_space_summary()

In [ ]:
    #just clean the sessio, recomendable if we execute some times the model.
    keras.backend.clear_session()

    # This callback saves the best model based in val_accuracy
    MCP = keras.callbacks.ModelCheckpoint(filepath='bestmodel.h5', monitor='val_accuracy', mode='auto', save_best_only=True, save_weights_only=False, verbose=1)
    RLP = keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=3, verbose=1, mode="auto", min_lr=0.000000001)
    ES = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto', patience=7, verbose=1) #min_delta=1e-4,

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

#creates a function that updates the learning rate based on the epoch number
def lr_update(epoch, lr):
    """
    For the first 5 epochs the learning rate will be 0.005.
    From epoch 6 and on, the learning rate will be reduced 1% per epoch
    """
    if epoch <= 5:
        # return 0.005
        return lr
    else:
        return lr * 0.99

lr_scheduler = LearningRateScheduler(lr_update)

In [ ]:
#Train the model
num_epochs = 30

history = tuner.search(train_generator,
          epochs=num_epochs, verbose=2,
          validation_data=validation_generator,
                       steps_per_epoch=17, validation_steps=8,callbacks=[lr_scheduler, MCP, keras.callbacks.TensorBoard("./tb_logs")]) #ES
                      #  callbacks=[ES, RLP, MCP, keras.callbacks.TensorBoard("./tb_logs")])

#trial 5
# Epoch 14: val_accuracy improved from 0.67578 to 0.87109, saving model to bestmodel.h5
# 17/17 - 25s - loss: 0.6426 - accuracy: 0.6534 - val_loss: 0.6852 - val_accuracy: 0.8711 - lr: 0.0092 - 25s/epoch - 1s/step
# Epoch 15/30

# Epoch 15: val_accuracy did not improve from 0.87109
# 17/17 - 25s - loss: 0.6347 - accuracy: 0.6573 - val_loss: 0.6838 - val_accuracy: 0.8398 - lr: 0.0091 - 25s/epoch - 1s/step

# **View logs on Tensorboard**

In [ ]:
#Code to see the results in Tensorboard

%load_ext tensorboard
# %reload_ext tensorboard

%tensorboard --logdir ./tb_logs

# **Query the results**

In [ ]:
from keras_tuner.engine.hyperparameters import HyperParameters

# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()


In [ ]:
# Return best 5 trials
best_trials = tuner.oracle.get_best_trials(num_trials=10)

for trial in best_trials:
    print("**********Trail id: ", trial.trial_id)
    trial.summary()
    print('\n')

In [ ]:
# After hyperparameter tuning, retrieve the best hyperparameters.
best_hp = tuner.get_best_hyperparameters()[0]
best_hp.values

# **Save the model**

In [ ]:
from pprint import pprint
pprint(best_hp.get_config(), compact=True)

In [ ]:
# save the model
best_model.save('./best/best_model.h5')
best_model.save('./best/best_model_new_version')

# Override the best model
load first K best models then we need to use tuner's get_best_models method as below

In [ ]:
# This will load 10 best hyper tuned models with the weights
# corresponding to their best checkpoint (at the end of the best epoch of best trial).
best_model_count = 10
bo_tuner_best_models = tuner.get_best_models(num_models=best_model_count).expect_partial()

Then you can access a specific best model as below:

In [ ]:
best_model_id = 1
override_model = bo_tuner_best_models[best_model_id]
override_model.summary()

This method is for querying the models trained during the search. For best performance, it is recommended to retrain your Model on the full dataset using the best hyperparameters found during search, which can be obtained using tuner.get_best_hyperparameters().

The best model according to me is second best model

In [ ]:
tuner_best_hyperparameters = tuner.get_best_hyperparameters(num_trials=10)
best_hp = tuner_best_hyperparameters[1]
model_override = tuner.hypermodel.build(best_hp)
best_hp.values

# **Visualize the model**

In [ ]:
# !pip install graphviz
from tensorflow.keras.utils import plot_model
plot_model(best_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# **Retrain the model(optional)**

In [ ]:
# Build the model with the best hp.
# model = build_model(best_hps)
model_override = tuner.hypermodel.build(best_hp)

num_epochs = 30
# Fit with the entire dataset.
history = model_override.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels),
                       callbacks=[ES, MCP, RLP], verbose=2)

In [ ]:
# Plot training results
plot_loss_acc(history)

In [ ]:
model_override.evaluate(testing_padded, testing_labels)